In [37]:
import numpy as np
np.random.seed(42)

# Input Collection

## Binary Class

In [38]:
def generate(filename,header=False,append_one=True):
    
    def splitter(line):
        var = line.split()
        x = [float(x) for x in var[:-1]]
        y = int(var[-1])
        return x,y
    
    file = open(filename)
    
    X = []
    Y = []
    
    lines = file.readlines()
    size = len(lines)
    
    if header:
        for i in range(1,size):
            x,y = splitter(lines[i])
            X.append(x)
            Y.append(y)
    else:
        for i in range(0,size):
            x,y = splitter(lines[i])
            X.append(x)
            Y.append(y)
    
    X = np.array(X)
    Y = np.array(Y)
    
    if append_one:
        X = np.insert(X,X.shape[1],values=1,axis=1)
    
    return X,Y

In [39]:
X_train,Y_train = generate('trainLinearlyNonSeparable.txt',header=True)
X_test,Y_test = generate('testLinearlyNonSeparable.txt',header=False)

## Basic Perceptron Algorithm

In [40]:
class BasicPerceptron:
    def __init__(self,low=-10,high=10,max_epoch=1000,lr=0.03):
        self.w = []
        self.low = low
        self.high = high
        self.max_epoch = max_epoch
        self.lr = lr
        pass
    
    def fit(self,X_train,Y_train):
        # setting random weight to weight vector
        self.w = np.random.uniform(self.low,self.high,(X_train.shape[1],1))
        
        for i in range(self.max_epoch):
            
            delta = np.zeros((X_train.shape[1],1))
            
            for instance,target in zip(X_train,Y_train):
                x = instance.reshape(-1,1)
                dot_product = np.dot(self.w.T,x).ravel()[0]
                
                if dot_product < 0 and target == 1: # only here is negative 
                    delta += x
                    continue
                
                if dot_product > 0 and target == 2:
                    delta -= x
                    continue
                
            self.w = self.w + self.lr*delta
        
        return self.w
    
    def predict(self,X,Y,verbose=True):
        correct = 0
        total = len(X)
        
        if verbose:
            print("Sample No","Feature Vector","Actual Class","Predicted Class")
        
        sample_no = 0
        
        predicted_class = -1
        for instance,true_class in zip(X,Y):
            x = instance.reshape(-1,1)
            dot_product = np.dot(self.w.T,x).ravel()[0]
            
            if dot_product > 0:
                predicted_class = 1
            if dot_product <0 :
                predicted_class = 2
            
            if verbose:
                print(sample_no,instance[:-1],true_class,predicted_class)
            
            if predicted_class == true_class:
                correct += 1
                
            sample_no += 1
        return correct/total

In [41]:
basicPerceptron = BasicPerceptron(low=-5,high=5)
basicPerceptron.fit(X_train,Y_train)

train_accuracy = basicPerceptron.predict(X_train,Y_train)
print("Training Accuracy: ",train_accuracy)

test_accuracy = basicPerceptron.predict(X_test,Y_test)
print("Test Accuracy: ",test_accuracy)

Sample No Feature Vector Actual Class Predicted Class
0 [2.62652986 3.49618671 5.53430073 7.98319422] 1 1
1 [ 3.36835433  8.45991344 11.65564447 15.14793504] 2 2
2 [1.84750112 4.03044984 5.11919415 8.06229515] 1 1
3 [2.20116595 3.44346774 5.93984785 7.99810615] 1 1
4 [ 4.14866737  8.82089133 12.34240521 15.97009586] 2 2
5 [ 3.45716704  7.66235563 12.19301316 16.34807149] 2 2
6 [2.6185906  3.80544923 6.09387247 8.47577694] 1 1
7 [2.52794267 4.39305442 6.50737322 7.86387989] 1 1
8 [ 4.33624659  8.12440622 11.60122942 15.85642983] 2 2
9 [ 4.47967649  7.22118832 11.7480347  16.08251303] 2 2
10 [ 3.54195649  7.81506285 12.00273979 15.62542458] 2 2
11 [ 4.4031082   8.17253829 11.14723811 16.60302882] 2 2
12 [ 4.78860959  8.51708366 12.15504186 17.64696703] 2 2
13 [1.74849265 4.3145575  6.25102373 8.48321221] 1 1
14 [2.40824725 4.28746473 5.63729111 8.14988272] 1 1
15 [1.69841992 4.48237659 5.37082943 7.7686641 ] 1 1
16 [1.66423374 3.80052942 5.88064063 7.9372555 ] 1 1
17 [1.91173104 4.748928

313 [2.59409059 4.18474236 6.44354863 8.06855218] 1 1
314 [ 4.64153029  6.7212372  11.09665696 15.70905984] 2 2
315 [2.23151898 4.51842156 6.68103472 7.08253998] 1 1
316 [ 4.62183599  8.92791981 12.02199614 15.31013633] 2 1
317 [3.51268718 3.57390619 6.28202966 7.8212276 ] 1 1
318 [2.06671449 4.93331908 5.63855009 8.54963623] 1 1
319 [2.84205973 4.3817151  6.35753865 8.62215006] 1 1
320 [ 3.81554455  7.88457627 11.74795306 16.52517246] 2 2
321 [2.42988389 4.43617616 5.23446579 7.67834782] 1 1
322 [ 3.61599291  7.2040681  11.87471421 15.85997361] 2 2
323 [2.3676503  4.46931385 5.52800548 8.11897499] 1 1
324 [1.80936719 4.18786993 6.69660512 7.36734228] 1 1
325 [ 4.00333919  8.2576492  11.15265118 16.5516213 ] 2 2
326 [ 4.15909255  9.21053997 12.42274778 16.3961848 ] 2 2
327 [ 4.77282027  7.84657561 12.58752214 15.92903057] 2 2
328 [ 4.29397455  8.07545885 11.85757277 15.50200178] 2 2
329 [ 4.00046412  8.05728612 12.02330026 16.01315881] 2 2
330 [ 3.89221356  8.28856562 12.37289725 16.42

258 [1.67704888 4.41692532 5.9439333  8.54878682] 1 1
259 [2.15715432 3.99752612 5.19363232 7.50095585] 1 1
260 [ 4.4588984   8.10796271 12.00973924 16.14943809] 2 2
261 [ 3.52153438  8.00104569 11.66041356 15.63626823] 2 2
262 [2.32965152 4.52286001 6.37073422 8.71252377] 1 1
263 [1.98027233 3.60617716 5.97731372 7.67060516] 1 1
264 [ 3.76730506  8.12978387 11.89470021 16.85359218] 2 2
265 [1.58653205 4.226155   5.78628112 7.70914407] 1 1
266 [1.84089073 4.1825303  5.13017732 8.0442204 ] 1 1
267 [1.66597419 3.73545406 6.48258917 8.83630211] 1 1
268 [1.15146953 3.70150542 6.08561875 8.04179009] 1 1
269 [ 4.31773028  8.0562072  11.55283131 16.1459349 ] 2 2
270 [1.36838124 3.76768001 6.54437686 7.77344157] 1 1
271 [ 4.51130797  8.36532865 12.11667285 16.36900332] 2 2
272 [2.08674116 4.33398183 6.31218613 7.33204346] 1 1
273 [2.26453673 3.36756018 6.28769733 8.49239294] 1 1
274 [ 3.92778487  7.43472635 11.22452392 16.36287958] 2 2
275 [1.12406242 4.12326938 5.72889027 7.91308817] 1 1
276 

## Reward and Punishment Algorithm

In [42]:
class RP_Perceptron:
    def __init__(self,low=-10,high=10,max_epoch=1000,lr=0.03):
        self.w = []
        self.low = low
        self.high = high
        self.max_epoch = max_epoch
        self.lr = lr
        pass
    
    def fit(self,X_train,Y_train):
        # setting random weight to weight vector
        self.w = np.random.uniform(self.low,self.high,(X_train.shape[1],1))
        
        for i in range(self.max_epoch):
            count = 0
            for instance,target in zip(X_train,Y_train):
                x = instance.reshape(-1,1)
                dot_product = np.dot(self.w.T,x).ravel()[0]
                
                if dot_product > 0 and target == 2:
                    count += 1
                    self.w = self.w - self.lr*x
                    
                if dot_product < 0 and target == 1:
                    count += 1
                    self.w = self.w + self.lr*x     
                    
            if count == 0:
                break
        
        return self.w
    
    def predict(self,X,Y,verbose=True):
        correct = 0
        total = len(X)
        
        if verbose:
            print("Sample No","Feature Vector","Actual Class","Predicted Class")
        
        sample_no = 0
        
        predicted_class = -1
        for instance,true_class in zip(X,Y):
            x = instance.reshape(-1,1)
            dot_product = np.dot(self.w.T,x).ravel()[0]
            
            if dot_product > 0:
                predicted_class = 1
            if dot_product < 0 :
                predicted_class = 2
                
            if verbose:
                print(sample_no,instance[:-1],true_class,predicted_class)
            
            if predicted_class == true_class:
                correct += 1
            
            sample_no += 1
        return correct/total

In [43]:
rp_Perceptron = RP_Perceptron(low=-5,high=5)
rp_Perceptron.fit(X_train,Y_train)
train_accuracy = rp_Perceptron.predict(X_train,Y_train)
print("Training Accuracy: ",train_accuracy)

test_accuracy = rp_Perceptron.predict(X_test,Y_test)
print("Test Accuracy: ",test_accuracy)

Sample No Feature Vector Actual Class Predicted Class
0 [2.62652986 3.49618671 5.53430073 7.98319422] 1 1
1 [ 3.36835433  8.45991344 11.65564447 15.14793504] 2 2
2 [1.84750112 4.03044984 5.11919415 8.06229515] 1 1
3 [2.20116595 3.44346774 5.93984785 7.99810615] 1 1
4 [ 4.14866737  8.82089133 12.34240521 15.97009586] 2 2
5 [ 3.45716704  7.66235563 12.19301316 16.34807149] 2 2
6 [2.6185906  3.80544923 6.09387247 8.47577694] 1 1
7 [2.52794267 4.39305442 6.50737322 7.86387989] 1 1
8 [ 4.33624659  8.12440622 11.60122942 15.85642983] 2 2
9 [ 4.47967649  7.22118832 11.7480347  16.08251303] 2 2
10 [ 3.54195649  7.81506285 12.00273979 15.62542458] 2 2
11 [ 4.4031082   8.17253829 11.14723811 16.60302882] 2 2
12 [ 4.78860959  8.51708366 12.15504186 17.64696703] 2 2
13 [1.74849265 4.3145575  6.25102373 8.48321221] 1 1
14 [2.40824725 4.28746473 5.63729111 8.14988272] 1 1
15 [1.69841992 4.48237659 5.37082943 7.7686641 ] 1 1
16 [1.66423374 3.80052942 5.88064063 7.9372555 ] 1 1
17 [1.91173104 4.748928

381 [1.87304381 3.99244335 6.76810899 8.66218991] 1 1
382 [1.25089521 5.05064176 6.16943913 7.75440269] 1 1
383 [ 3.88277613  7.75048556 11.70601136 16.52412262] 2 2
384 [ 4.92977465  8.63151444 12.29898137 16.42447019] 2 2
385 [1.57544998 4.80746949 6.83283517 7.33456519] 1 1
386 [1.63531903 4.78362622 5.94446834 7.85249782] 1 1
387 [ 2.87179735  8.00411691 11.75736781 15.74533347] 2 2
388 [1.24909927 4.2662285  7.42534669 7.73258643] 1 1
389 [1.97338716 4.21042272 5.6503092  8.28995679] 1 1
390 [2.02150248 3.71770694 5.97967851 8.35010815] 1 1
391 [ 3.54215953  8.07697658 12.04414474 15.59830081] 2 2
392 [2.63410582 3.38087939 6.38062189 7.55967413] 1 1
393 [ 3.90339134  8.55351926 12.14784497 15.08049317] 2 2
394 [ 3.59775091  7.5526482  11.54269502 16.21577333] 2 2
395 [ 4.10279381  7.33916465 13.67673019 16.06149671] 2 2
396 [ 3.06923648  7.90953017 11.70216588 15.59492431] 2 2
397 [ 3.57120806  8.6463275  12.18323558 15.99709956] 2 2
398 [1.94212281 4.02586627 5.79610626 7.486333

230 [ 3.58566618  8.61003124 11.54033325 15.84124854] 2 2
231 [ 3.38587605  8.4328512  12.57303655 15.51604918] 2 2
232 [ 5.15139878  7.85818983 11.8873585  16.53214052] 2 2
233 [1.15854927 4.11376034 6.34101906 8.10927163] 1 1
234 [2.19178512 4.3729198  5.88492646 8.01306787] 1 1
235 [1.3571285  4.11247461 6.14667251 8.47624098] 1 1
236 [ 4.36702284  7.56624851 12.25864696 16.61510461] 2 2
237 [ 3.53925239  8.46355669 12.8513296  15.80931288] 2 2
238 [2.25777737 4.07496509 5.75828455 7.84072998] 1 1
239 [ 3.68875426  7.28867757 11.55498922 15.20752083] 2 2
240 [ 4.2430978   8.72822198 11.81245755 15.65745084] 2 2
241 [ 4.19134742  8.14797283 11.54585487 15.87088761] 2 2
242 [ 4.39747476  8.42903135 12.25592086 15.70498986] 2 2
243 [ 3.45245221  8.04650062 12.24553101 15.78674623] 2 2
244 [2.61275742 4.38510261 5.93436986 8.23553716] 1 1
245 [2.18211999 3.53351366 5.46918865 8.70841052] 1 1
246 [ 4.16661647  8.03759185 11.84120567 15.4473556 ] 2 2
247 [ 4.26077796  7.86382769 11.530196

## Pocket Algorithm

In [44]:
class Pocket_Perceptron:
    def __init__(self,low=-10,high=10,max_epoch=1000,lr=0.03):
        self.w = []
        self.low = low
        self.high = high
        self.max_epoch = max_epoch
        self.lr = lr
        self.wp = []
        pass
    
    def fit(self,X_train,Y_train):
        # setting random weight to weight vector
        self.w = np.random.uniform(self.low,self.high,(X_train.shape[1],1))
        self.wp = self.w
        misclassification = len(X_train)
        
        for i in range(self.max_epoch):
            count = 0
            
            delta = np.zeros((X_train.shape[1],1))
            
            for instance,target in zip(X_train,Y_train):
                x = instance.reshape(-1,1)
                dot_product = np.dot(self.w.T,x).ravel()[0]
                
                if dot_product < 0 and target == 1: # only here is negative 
                    delta += x
                    count += 1
                    continue
                
                if dot_product > 0 and target == 2:
                    delta -= x
                    count += 1
                    continue
            
            if count < misclassification:
                misclassification = count
                self.wp = self.w
            
            self.w = self.w + self.lr*delta
            
        return self.wp
    
    def predict(self,X,Y,verbose=True):
        correct = 0
        total = len(X)
        
        predicted_class = -1
        sample_no = 0
        
        if verbose:
            print("Sample No","Feature Vector","Actual Class","Predicted Class")
        
        for instance,true_class in zip(X,Y):
            x = instance.reshape(-1,1)
            dot_product = np.dot(self.wp.T,x).ravel()[0]
            
            if dot_product >= 0:
                predicted_class = 1
            if dot_product <0 :
                predicted_class = 2
                
            if verbose:
                print(sample_no,instance[:-1],true_class,predicted_class)
            
            if predicted_class == true_class:
                correct += 1
                
            sample_no += 1
        return correct/total

In [45]:
pocket_Perceptron = Pocket_Perceptron(low=-5,high=5)
pocket_Perceptron.fit(X_train,Y_train)
train_accuracy = pocket_Perceptron.predict(X_train,Y_train)
print("Training Accuracy: ",train_accuracy)

test_accuracy = pocket_Perceptron.predict(X_test,Y_test)
print("Test Accuracy: ",test_accuracy)

Sample No Feature Vector Actual Class Predicted Class
0 [2.62652986 3.49618671 5.53430073 7.98319422] 1 1
1 [ 3.36835433  8.45991344 11.65564447 15.14793504] 2 2
2 [1.84750112 4.03044984 5.11919415 8.06229515] 1 1
3 [2.20116595 3.44346774 5.93984785 7.99810615] 1 1
4 [ 4.14866737  8.82089133 12.34240521 15.97009586] 2 2
5 [ 3.45716704  7.66235563 12.19301316 16.34807149] 2 2
6 [2.6185906  3.80544923 6.09387247 8.47577694] 1 1
7 [2.52794267 4.39305442 6.50737322 7.86387989] 1 1
8 [ 4.33624659  8.12440622 11.60122942 15.85642983] 2 2
9 [ 4.47967649  7.22118832 11.7480347  16.08251303] 2 2
10 [ 3.54195649  7.81506285 12.00273979 15.62542458] 2 2
11 [ 4.4031082   8.17253829 11.14723811 16.60302882] 2 2
12 [ 4.78860959  8.51708366 12.15504186 17.64696703] 2 2
13 [1.74849265 4.3145575  6.25102373 8.48321221] 1 1
14 [2.40824725 4.28746473 5.63729111 8.14988272] 1 1
15 [1.69841992 4.48237659 5.37082943 7.7686641 ] 1 1
16 [1.66423374 3.80052942 5.88064063 7.9372555 ] 1 1
17 [1.91173104 4.748928

25 [ 3.70042491  7.72294829 12.8119886  15.40186774] 2 2
26 [ 4.55134399  8.52194082 11.40351591 15.31426346] 2 2
27 [2.30910517 3.52978264 6.20564743 7.90608131] 1 1
28 [ 4.57062399  8.54269888 12.0601304  16.64034094] 2 2
29 [ 3.68819451  8.50505699 11.11906041 16.18142737] 2 2
30 [1.91293053 4.21906644 6.09826201 7.84933041] 1 1
31 [1.34780487 3.7026161  5.66813078 7.69390057] 1 1
32 [ 3.89265938  7.88145475 11.65986642 16.68607458] 2 2
33 [ 3.70993328  7.31609266 11.90165185 16.93276685] 2 2
34 [ 3.00815588  7.61346918 11.61118618 15.01532173] 2 2
35 [ 3.70988571  8.16423199 12.80868772 15.81975738] 2 2
36 [1.90108789 4.52542133 6.08688606 8.1555754 ] 1 1
37 [ 3.80709491  9.01786471 12.89272625 15.40763365] 2 2
38 [ 3.88699831  7.87040379 12.4818662  15.73506274] 2 2
39 [2.33200183 3.34659082 6.03560741 8.20158032] 1 1
40 [1.35431657 3.77160188 6.47433526 9.05717391] 1 1
41 [2.21431834 4.09532018 6.70221592 8.16956455] 1 1
42 [ 3.58087722  7.92349614 11.44099638 16.224739  ] 2 2
43

269 [ 4.31773028  8.0562072  11.55283131 16.1459349 ] 2 2
270 [1.36838124 3.76768001 6.54437686 7.77344157] 1 1
271 [ 4.51130797  8.36532865 12.11667285 16.36900332] 2 2
272 [2.08674116 4.33398183 6.31218613 7.33204346] 1 1
273 [2.26453673 3.36756018 6.28769733 8.49239294] 1 1
274 [ 3.92778487  7.43472635 11.22452392 16.36287958] 2 2
275 [1.12406242 4.12326938 5.72889027 7.91308817] 1 1
276 [1.79523324 3.8739797  6.82137184 8.4436377 ] 1 1
277 [2.51593741 4.40165506 5.90494461 7.56403837] 1 1
278 [ 4.25617975  6.79405872 12.81580723 16.14245234] 2 2
279 [ 4.41669486  8.28354984 11.78949583 15.13007543] 2 2
280 [2.35541848 3.19455986 5.63843997 8.08797384] 1 1
281 [1.93669184 3.96859395 6.92495151 7.80090182] 1 1
282 [ 4.51563412  8.19819    12.51038645 16.05563503] 2 2
283 [ 3.40603276  7.80312347 12.18102739 15.91157124] 2 2
284 [1.80520771 2.51624435 6.3764042  7.80635029] 1 1
285 [2.2626572  4.24525787 6.20787679 8.59285372] 1 1
286 [2.5454942  4.40333553 4.96158565 7.6547179 ] 1 1


# Multi Class Perceptron
## Kesler’s Construction

In [46]:
X_train,Y_train = generate('multiclass_train.txt',header=True)
X_test,Y_test = generate('multiclass_test.txt',header=False)

In [47]:
class KeslarConstruction:
    def __init__(self,low=-10,high=10,max_epoch=1000,lr=0.03):
        self.w = [] # contains array of weights
        self.low = low
        self.high = high
        self.max_epoch = max_epoch
        self.lr = lr
        self.number_of_classes = 0
        pass
    
    def fit(self,X_train,Y_train):
        unique_classes = np.unique(Y_train)
        
        self.number_of_classes = len(unique_classes)
        for i in range(self.number_of_classes):
            self.w.append(np.random.uniform(self.low,self.high,(X_train.shape[1],1)))
        self.w = np.array(self.w)
        
        dataset = []
        
        for instance,target in zip(X_train,Y_train):
            current_class = target-1
            for each in range(self.number_of_classes):
                if current_class!= each:
                    instance = instance.reshape(-1,1)
                    row = np.zeros((self.number_of_classes,X_train.shape[1],1))
                    row[current_class] = instance
                    row[each] = -1*instance
                    dataset.append(row)
        
        dataset = np.array(dataset)
        N = len(dataset)
    
        for _ in range(self.max_epoch):
            count = 0
            for i in range(N):
                dot_product = np.dot(self.w.ravel(),dataset[i].ravel())
                if dot_product <= 0:
                    self.w = self.w + self.lr*dataset[i] # punishment
                    count += 1
                else:
                    pass # reward
            if count == 0:
                break
    
    def predict(self,X,Y):
        correct = 0
        total = len(X)
        
        for instance,true_class in zip(X,Y):
            max = -10e10
            predicted_class = -1
            for each in range(self.number_of_classes):
                dot_product = np.dot(self.w[each].T,instance.reshape(-1,1)).ravel()
                if dot_product[0] > max:
                    max = dot_product[0]
                    predicted_class = each+1
            
            if true_class == predicted_class:
                correct += 1
                
        return correct/total

In [48]:
kc = KeslarConstruction()
kc.fit(X_train,Y_train)
train_accuracy = kc.predict(X_train,Y_train)
test_accuracy = kc.predict(X_test,Y_test)

print(train_accuracy,test_accuracy)

1.0 0.98
